In [1]:
from coffea.nanoevents import NanoEventsFactory, BaseSchema, PFNanoAODSchema
import json
import fastjet
import numpy as np
import awkward as ak
from coffea import processor
import hist
import coffea.nanoevents.methods.vector as vector
import warnings
import hist.dask as dhist
import dask
import pickle
import os
import dask_awkward as dak

In [2]:
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")
warnings.filterwarnings("ignore", "Please ensure")

In [3]:
import coffea
print(coffea.__version__)
print(ak.__version__)
print(dak.__version__)

2024.1.1
2.5.2
2024.1.1


In [4]:
with open('../filelists/hbb_files.txt', 'r') as f:
    hbb_files = [line.strip() for line in f]
hbb = NanoEventsFactory.from_root(
    {'/project01/ndcms/cmoore24/signal/hbb/' + hbb_files[0]: "/Events"},
    #permit_dask=True,
    schemaclass=PFNanoAODSchema,
    metadata={"dataset": "Hbb"},
).events()

In [5]:
fatjet = hbb.FatJet
cut = ((fatjet.pt > 300) & (fatjet.msoftdrop > 110) & 
       (fatjet.msoftdrop < 140) & (abs(fatjet.eta) < 2.4)) #& (fatjet.btagDDBvLV2 > 0.89)
slimmed = hbb.FatJet[cut]

In [6]:
def color_ring(fatjet, variant=False, groomed=False):
    pf = ak.flatten(fatjet.constituents.pf, axis=1)
    if groomed==True:
        jetdef = fastjet.JetDefinition(fastjet.cambridge_algorithm, 0.8)
        cluster = fastjet.ClusterSequence(pf, jetdef)
        softdrop = cluster.exclusive_jets_softdrop_grooming()
        pf = softdrop.constituents
    jetdef = fastjet.JetDefinition(fastjet.cambridge_algorithm, 0.2)
    cluster = fastjet.ClusterSequence(pf, jetdef)
    #subjets = cluster.exclusive_subjets_up_to(data=cluster.exclusive_jets(n_jets=1), nsub=3) #legacy
    subjets = cluster.inclusive_jets()
    vec = ak.zip({
        "x": subjets.px,
        "y": subjets.py,
        "z": subjets.pz,
        "t": subjets.E,
        },
        with_name = "LorentzVector",
        behavior=vector.behavior,
        )
    vec = ak.pad_none(vec, 3)
    vec["norm3"] = np.sqrt(vec.dot(vec))
    vec["idx"] = ak.local_index(vec)
    # i, j = ak.unzip(ak.combinations(vec, 2))
    # best = ak.argmax((i + j).mass, axis=1, keepdims=True)
    # leg1, leg2 = ak.firsts(i[best]), ak.firsts(j[best])
    # leg3 = ak.firsts(vec[(vec.idx != leg1.idx) & (vec.idx != leg2.idx)]) #new
    i, j, k = ak.unzip(ak.combinations(vec, 3))
    best = ak.argmin(abs((i + j + k).mass - 125), axis=1, keepdims=True)
    order_check = ak.concatenate([i[best].mass, j[best].mass, k[best].mass], axis=1)
    largest = ak.argmax(order_check, axis=1, keepdims=True)
    smallest = ak.argmin(order_check, axis=1, keepdims=True)
    leading_particles = ak.concatenate([i[best], j[best], k[best]], axis=1)
    leg1 = leading_particles[largest]
    leg3 = leading_particles[smallest]
    leg2 = leading_particles[(leading_particles.idx != ak.flatten(leg1.idx)) & (leading_particles.idx != ak.flatten(leg3.idx))]
    leg1 = ak.firsts(leg1)
    leg2 = ak.firsts(leg2)
    leg3 = ak.firsts(leg3)
    a12 = np.arccos(leg1.dot(leg2) / (leg1.norm3 * leg2.norm3))
    a13 = np.arccos(leg1.dot(leg3) / (leg1.norm3 * leg3.norm3))
    a23 = np.arccos(leg2.dot(leg3) / (leg2.norm3 * leg3.norm3))
    if variant == False:
        color_ring = ((a13**2 + a23**2)/(a12**2))
    else: 
        color_ring = a13**2 + a23**2 - a12**2
    return color_ring

In [7]:
uf_cr = ak.unflatten(color_ring(slimmed), counts=ak.num(slimmed))

In [8]:
uf_cr

dask.awkward<unflatten, npartitions=1>

In [9]:
dak.necessary_columns(uf_cr)

{'from-uproot-eb097cdb8f255639280992ed28d8dcb4': frozenset({'FatJetPFCands_pFCandsIdx',
            'FatJet_eta',
            'FatJet_msoftdrop',
            'FatJet_nConstituents',
            'FatJet_pt',
            'PFCands_eta',
            'PFCands_mass',
            'PFCands_phi',
            'PFCands_pt',
            'nFatJet',
            'nFatJetPFCands',
            'nPFCands'})}

In [11]:
a = ak.flatten(uf_cr.compute())

In [12]:
a

<Array [2, 1.39, 0.813, 0.935, ..., 17.6, 2.58, 1.44] type='889 * ?float64'>